In [13]:
import cv2
import numpy as np
from numpy import uint8,float32,float64
import matplotlib.pyplot as plt

from skimage.io import imread,imsave,imshow
from skimage.color import rgb2gray
import sys
import os

import tkinter as tk

import tkinter.ttk as ttk
from PIL import Image, ImageTk
from tkinter import messagebox as tkMessageBox
from tkinter import filedialog as tkFileDialog

In [40]:
##メイン処理,関数をすべて読み込んでから実行
#画像読み込み及び変換
defultim_row=cv2.imread('image/girl.jpg')
defultim=cv2.cvtColor(defultim_row, cv2.COLOR_BGR2GRAY)
furieIm_row=imageChenge_furie(defultim)
furieIm=imFurieShow( furieIm_row)
rePlusIm=np.zeros(furieIm.shape)
clickIm=np.zeros(furieIm.shape)
reNowIm=np.zeros(furieIm.shape)

rePlusIm=rePlusIm.astype(complex)
clickIm=clickIm.astype(complex)
reNowIm=reNowIm.astype(complex)

#windowの作成
root=tk.Tk()
root.title("Application")
root.geometry('800x600')
root.resizable(0, 0)

root.grid_columnconfigure(0, weight=1)
root.grid_rowconfigure(0, weight=1)

mainFrame = tk.Frame(root, bd=5, relief="ridge")
#リセットボタン
label_button=tk.Button(mainFrame,text="reset",width=10
                       ,command=reset_im)

#imageのLabelの作成
label_image_defult=tk.Label(mainFrame)
label_image_furie=tk.Label(mainFrame)
label_image_click=tk.Label(mainFrame)
label_image_rePlus=tk.Label(mainFrame)
label_image_reNow=tk.Label(mainFrame)

#画像の初期値の設定
chenge_frameImage(label_image_defult,defultim)
chenge_frameImage(label_image_furie,furieIm)
chenge_frameImage(label_image_rePlus, np.abs(rePlusIm))
chenge_frameImage(label_image_click,np.abs(clickIm))
chenge_frameImage(label_image_reNow,np.abs(reNowIm))
#画像クリック時の命令
label_image_click.bind("<Button-1>",img_click)
label_image_click.bind("<B1-Motion>",img_click)
#配置
mainFrame.grid(row=0,column=0,stick="nwse")
mainFrame.grid_columnconfigure((0, 2), weight=1)
mainFrame.grid_rowconfigure((0, 1), weight=1)

label_image_defult.grid(row=0,column=0)
label_image_furie.grid(row=0,column=1)
label_button.grid(row=0,column=2)
label_image_rePlus.grid(row=1,column=0)
label_image_click.grid(row=1,column=1)
label_image_reNow.grid(row=1,column=2)

root.mainloop()

C:\Users\seiji\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
def chenge_frameImage(frame,im):#画像の更新処理
    
    imf=Image.fromarray(im)
    imm=ImageTk.PhotoImage(imf,master=mainFrame)
    frame.configure(image=imm)
    frame.img=imm
def chenge_frameImage_complex(frame,im):#値が非常に小さい場合の更新前の変換処理
    imm=MainHenkan(np.abs(im))
    chenge_frameImage(frame,imm)

In [23]:
def img_click(ev):#画像クリック時の変換処理
        area=1#フーリエ変換を適用する範囲
        
        #画像の変換処理
        global clickIm
        clickIm[ev.y:ev.y+area,ev.x:ev.x+area]=furieIm_row[ev.y:ev.y+area,ev.x:ev.x+area]
        global reNowIm
        reNowIm=imageChenge_sinWaveNow(furieIm_row,ev.x,ev.y)
        global rePlusIm
        rePlusIm=imageChenge_sinWavePlus(clickIm)
        
        
        #画像の更新
        global label_image_rePlus
        global label_image_click
        global label_image_reNow
        chenge_frameImage_complex(label_image_rePlus,rePlusIm)
        chenge_frameImage(label_image_click, np.abs(clickIm))
        chenge_frameImage_complex(label_image_reNow,reNowIm)
def reset_im():#リセットボタンの処理
    global clickIm
    global reNowIm
    global rePlusIm
    clickIm=np.zeros(furieIm.shape)
    reNowIm=np.zeros(furieIm.shape)
    rePlusIm=np.zeros(furieIm.shape)
    clickIm=clickIm.astype(complex)
    reNowIm=reNowIm.astype(complex)
    rePlusIm=rePlusIm.astype(complex)
    
    chenge_frameImage(label_image_rePlus,np.abs(rePlusIm))
    chenge_frameImage(label_image_click, np.abs(clickIm))
    chenge_frameImage(label_image_reNow,np.abs(reNowIm))

In [16]:
def imageChenge_furie(im):
    #画像のフーリエ変換
    im=Image.fromarray(im)
    f=np.fft.fft2(im)
    return np.fft.fftshift(f)
def imFurieShow(im):#返還後の調整
    return 20*np.log(np.abs(im))
def imageChenge_furieR(im):
    #画像のフーリエ逆変換
    f=np.fft.ifftshift(im)
    f=np.fft.ifft2(f)
    return np.abs(f)

In [25]:
def imageChenge_sinWaveNow(im_furie,x,y):
    #サイン波への変換、特定座標のみ
    zero=np.zeros(im_furie.shape)
    zero=zero.astype(complex)
    area=1#フーリエ変換の対象範囲
    zero[y:y+area,x:x+area]=im_furie[y:y+area,x:x+area]
    i=np.fft.ifftshift(zero)
    i=np.fft.ifft2(i)
    return i
def imageChenge_sinWavePlus(im_click):
    #サイン波への変換。合計
    f=np.fft.ifftshift(im_click)
    f=np.fft.ifft2(f)
    return f

In [24]:
#フーリエ変換後の画像を見えやすくする処理
def MainHenkan(im):#採用したもの。よくわからないがうまくいく
    return (im-np.min(im))*255/(np.max(im)-np.min(im))
def Contrast(im,E):#コントラスト強調
    e=0.0001
    M=np.median(im)
    return 1/(1+(M/(im+e))**E)
def dynamin(im):#フーリエ変換にはこれがいいらしいがうまくいかなかった
    imRange=np.max(im)-np.min(im)
    print(imRange)    
    print(np.max(im))
    print(np.min(im))

    return (1/(np.log(1+imRange)))*(np.log(1+im))